In [1]:
import pandas as pd
# Read the CSV file into a DataFrame
data = pd.read_excel('/content/drive/MyDrive/walmart project/Online Retail.xlsx')

In [2]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
# Calculate total revenue
data['TotalRevenue'] = data['Quantity'] * data['UnitPrice']

# Monthly Revenue Trend
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['Month'] = data['InvoiceDate'].dt.to_period('M')
monthly_revenue = data.groupby('Month')['TotalRevenue'].sum()

# Top-selling products
top_products = data.groupby('Description')['Quantity'].sum().nlargest(10)

# Customer Loyalty
loyal_customers = data.groupby('CustomerID')['InvoiceDate'].nunique().reset_index()
loyal_customers = loyal_customers[loyal_customers['InvoiceDate'] > 1]

# Average Order Value
avg_order_value = data.groupby('CustomerID')['TotalRevenue'].mean()

In [5]:
# RFM Analysis
recency = data.groupby('CustomerID')['InvoiceDate'].max().reset_index()
recency['Recency'] = (data['InvoiceDate'].max() - recency['InvoiceDate']).dt.days
frequency = data.groupby('CustomerID')['InvoiceDate'].nunique().reset_index()
monetary = data.groupby('CustomerID')['TotalRevenue'].sum().reset_index()
rfm_data = pd.merge(recency, frequency, on='CustomerID')
rfm_data = pd.merge(rfm_data, monetary, on='CustomerID')


In [6]:

# Cluster Analysis (Example: K-means clustering)
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Select features for clustering
cluster_data = data.groupby('CustomerID').agg({
    'Quantity': 'sum',
    'UnitPrice': 'mean',
    'TotalRevenue': 'mean'
}).reset_index()

# Normalize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(cluster_data[['Quantity', 'UnitPrice', 'TotalRevenue']])

# Perform K-means clustering
kmeans = KMeans(n_clusters=4, random_state=0)
cluster_data['Cluster'] = kmeans.fit_predict(scaled_data)

# Cohort Analysis
data['InvoiceMonth'] = data['InvoiceDate'].dt.to_period('M')
cohort_data = data.groupby(['InvoiceMonth', 'CustomerID']).agg({
    'InvoiceDate': 'nunique',
    'TotalRevenue': 'sum'
})
cohort_data = cohort_data.reset_index()
cohort_data['CohortMonth'] = cohort_data.groupby('CustomerID')['InvoiceMonth'].transform('min')

# Perform further cohort analysis based on cohorts and metrics of interest

# Print the results
print("Total Revenue:")
print(data['TotalRevenue'].sum())



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Total Revenue:
9747747.933999998


In [7]:
print("Monthly Revenue Trend:")
monthly_revenue

Monthly Revenue Trend:


Month
2010-12     748957.020
2011-01     560000.260
2011-02     498062.650
2011-03     683267.080
2011-04     493207.121
2011-05     723333.510
2011-06     691123.120
2011-07     681300.111
2011-08     682680.510
2011-09    1019687.622
2011-10    1070704.670
2011-11    1461756.250
2011-12     433668.010
Freq: M, Name: TotalRevenue, dtype: float64

In [8]:
print("\nTop-selling Products:\n")
top_products


Top-selling Products:



Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     53847
JUMBO BAG RED RETROSPOT               47363
ASSORTED COLOUR BIRD ORNAMENT         36381
POPCORN HOLDER                        36334
PACK OF 72 RETROSPOT CAKE CASES       36039
WHITE HANGING HEART T-LIGHT HOLDER    35317
RABBIT NIGHT LIGHT                    30680
MINI PAINT SET VINTAGE                26437
PACK OF 12 LONDON TISSUES             26315
PACK OF 60 PINK PAISLEY CAKE CASES    24753
Name: Quantity, dtype: int64

In [9]:
print("\nLoyal Customers:\n")
loyal_customers


Loyal Customers:



,CustomerID,InvoiceDate
0,12346.0,2
1,12347.0,7
2,12348.0,4
5,12352.0,11
9,12356.0,3
...,...,...
4364,18276.0,3
4365,18277.0,2
4369,18282.0,3
4370,18283.0,16


In [10]:
print("\nAverage Order Value:\n")
avg_order_value


Average Order Value:



CustomerID
12346.0     0.000000
12347.0    23.681319
12348.0    57.975484
12349.0    24.076027
12350.0    19.670588
             ...    
18280.0    18.060000
18281.0    11.545714
18282.0    13.584615
18283.0     2.771005
18287.0    26.246857
Name: TotalRevenue, Length: 4372, dtype: float64

In [11]:
print("\nRFM Analysis:\n")
rfm_data


RFM Analysis:



,CustomerID,InvoiceDate_x,Recency,InvoiceDate_y,TotalRevenue
0,12346.0,2011-01-18 10:17:00,325,2,0.00
1,12347.0,2011-12-07 15:52:00,1,7,4310.00
2,12348.0,2011-09-25 13:13:00,74,4,1797.24
3,12349.0,2011-11-21 09:51:00,18,1,1757.55
4,12350.0,2011-02-02 16:01:00,309,1,334.40
...,...,...,...,...,...
4367,18280.0,2011-03-07 09:52:00,277,1,180.60
4368,18281.0,2011-06-12 10:53:00,180,1,80.82
4369,18282.0,2011-12-02 11:43:00,7,3,176.60
4370,18283.0,2011-12-06 12:02:00,3,16,2094.88


In [12]:
print("\nCluster Analysis:\n")
cluster_data


Cluster Analysis:



,CustomerID,Quantity,UnitPrice,TotalRevenue,Cluster
0,12346.0,0,1.040000,0.000000,0
1,12347.0,2458,2.644011,23.681319,0
2,12348.0,2341,5.764839,57.975484,0
3,12349.0,631,8.289041,24.076027,0
4,12350.0,197,3.841176,19.670588,0
...,...,...,...,...,...
4367,18280.0,45,4.765000,18.060000,0
4368,18281.0,54,5.622857,11.545714,0
4369,18282.0,98,4.821538,13.584615,0
4370,18283.0,1397,1.614987,2.771005,0


In [13]:
print("\nCohort Analysis:\n")
cohort_data


Cohort Analysis:



,InvoiceMonth,CustomerID,InvoiceDate,TotalRevenue,CohortMonth
0,2010-12,12347.0,1,711.79,2010-12
1,2010-12,12348.0,1,892.80,2010-12
2,2010-12,12370.0,2,1868.02,2010-12
3,2010-12,12377.0,1,1001.52,2010-12
4,2010-12,12383.0,1,600.72,2010-12
...,...,...,...,...,...
13670,2011-12,18245.0,1,894.25,2010-12
13671,2011-12,18272.0,1,367.88,2011-04
13672,2011-12,18273.0,1,51.00,2011-03
13673,2011-12,18282.0,1,77.84,2011-08
